In [1]:
!pwd

/home/hello/PycharmProjects/nosql_webscrapper_fastapi/nbs


In [2]:
cd ..


/home/hello/PycharmProjects/nosql_webscrapper_fastapi


In [3]:
import requests
from slugify import slugify
from requests_html import HTML
from app import scrapper
import pprint
import re


In [4]:
url = "https://www.amazon.com/Vitamix-Explorian-Professional-Grade-Low-Profile-Container/dp/B07CX95VRT/ref=sr_1_3?keywords=Vitamix&qid=1669195517&sr=8-3"


In [5]:
scr = scrapper.Scraper(url=url, endless_scroll=True)
html_str = scr.get()
html_obj = HTML(html=html_str)
def extract_element_text(html_obj, element_id):
    el = html_obj.find(element_id, first=True)
    if not el:
        return ""
    return el.text

In [6]:
price = extract_element_text(html_obj, '#corePrice_desktop > div > table > tbody > tr:nth-child(2) > td.a-span12 > span.a-price.a-text-price.a-size-medium.apexPriceToPay > span.a-offscreen')
price

'$289.95'

In [7]:
title = extract_element_text(html_obj, "#productTitle")
def extract_tables(html_obj):
    return html_obj.find("table")



In [8]:
tables = extract_tables(html_obj)
tables

[<Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-lineitem', 'a-align-top')>,
 <Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-normal', 'a-spacing-micro')>,
 <Element 'table' id='HLCXComparisonTable' class=('a-bordered', 'a-horizontal-stripes', 'a-spacing-none', 'a-size-base', 'comparison_table')>,
 <Element 'table' id='productDetails_detailBullets_sections1' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' id='productDetails_feedback_sections' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' id='histogramTable' class=('a-normal', 'a-spacing-none') role='presentation'>,
 <Element 'table' id='histogramTable' class=('a-normal', 'a-align-center', 'a-spacing-base')>,
 <Element 'table' class=('rhf-loading-middle',)>,
 <Element 'table' class=('navFooterMoreOnAmazon',) cellspacing='0'>]

In [9]:
def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    val = None
    if len(x) == 1:
        val = x[0]
    return val
    

In [10]:
def extract_dataset(tables):
    dataset = {}
    for table in tables:
        for tbody in table.element.getchildren():
            for tr in tbody.getchildren():
                row = []
                for col in tr.getchildren():
                    content = ""
                    try:
                        content = col.text_content()
                    except:
                        pass
                    if content != "":
                        _content = content.strip()
                        row.append(_content)
                if len(row) != 2:
                    continue
                key = row[0]
                value = row[1]

                data = {}
                key = slugify(key)
                if key in dataset:
                    continue
                else:
                    if "$" in value:
                        new_key = key
                        old_key = f'{key}_raw'
                        new_value = extract_price_from_string(value)
                        old_value = value
                        dataset[new_key] = new_value
                        dataset[old_key] = old_value
                    else:
                        dataset[key] = value
    return dataset


In [11]:
dataset = extract_dataset(tables)
pprint.pprint(dataset)

{'asin': 'B07CX95VRT',
 'best-sellers-rank': '#2,942 in Kitchen & Dining (See Top 100 in Kitchen & '
                      'Dining)  #17 in Countertop Blenders',
 'blade-material': 'Stainless Steel',
 'brand': 'Vitamix',
 'capacity': '64 Fluid Ounces',
 'color': 'Black',
 'country-of-origin': 'USA',
 'customer-reviews': '/* \n'
                     '    * Fix for UDP-1061. Average customer reviews has a '
                     'small extra line on hover \n'
                     '    * '
                     'https://omni-grok.amazon.com/xref/src/appgroup/websiteTemplates/retail/SoftlinesDetailPageAssets/udp-intl-lock/src/legacy.css?indexName=WebsiteTemplates#40\n'
                     '    */\n'
                     '    .noUnderline a:hover { \n'
                     '        text-decoration: none; \n'
                     '    }\n'
                     '\n'
                     '\n'
                     ' \n'
                     '  4.7 out of 5 stars    \n'
                     '    